In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dfPedBarrios = pd.read_excel("Pedidos x barrio ene16-abr18.xlsx")
dfPedBarrios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
Barrio/Localidad    10000 non-null object
Fecha               10000 non-null datetime64[ns]
Dia                 10000 non-null object
Turno               10000 non-null object
Local               10000 non-null object
Pedidos             10000 non-null int64
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 468.8+ KB


In [4]:
dfPron = pd.read_excel("Pronosticos ene16 a abr18.xlsx")
dfPron.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 4 columns):
fecha          851 non-null datetime64[ns]
Soler Dia      834 non-null float64
Soler Noche    834 non-null float64
Totales        851 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 26.7 KB


In [6]:
dfTotPedidos = dfPedBarrios.groupby("Fecha").sum()
dfTotPedidos.head()

,Pedidos
Fecha,
2016-01-01,74
2016-01-02,80
2016-01-03,79
2016-01-04,52
2016-01-05,63


In [7]:
dfTotPedidos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 846 entries, 2016-01-01 to 2018-04-30
Data columns (total 1 columns):
Pedidos    846 non-null int64
dtypes: int64(1)
memory usage: 13.2 KB


## Integrando con las predicciones a una misma BD

In [11]:
dfPron.set_index("fecha", inplace=True)
dfPron.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 851 entries, 2016-01-01 to 2018-04-30
Data columns (total 3 columns):
Soler Dia      834 non-null float64
Soler Noche    834 non-null float64
Totales        851 non-null int64
dtypes: float64(2), int64(1)
memory usage: 26.6 KB


In [21]:
df = dfTotPedidos.join(dfPron, rsuffix="Pred_")
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 846 entries, 2016-01-01 to 2018-04-30
Data columns (total 4 columns):
Pedidos        846 non-null int64
Soler Dia      829 non-null float64
Soler Noche    829 non-null float64
Totales        846 non-null int64
dtypes: float64(2), int64(2)
memory usage: 73.0 KB


In [22]:
df.head()

,Pedidos,Soler Dia,Soler Noche,Totales
Fecha,,,,
2016-01-01,74,0.0,70.0,70
2016-01-02,80,0.0,79.0,79
2016-01-03,79,0.0,70.0,70
2016-01-04,52,0.0,52.0,52
2016-01-05,63,0.0,55.0,55


In [26]:
#cambiando nombre columnas
df.columns = ["PedidosReales", "SolerDiaPred", "SolerNochePred", "TotalesPred"]
df.head()

,PedidosReales,SolerDiaPred,SolerNochePred,TotalesPred
Fecha,,,,
2016-01-01,74,0.0,70.0,70
2016-01-02,80,0.0,79.0,79
2016-01-03,79,0.0,70.0,70
2016-01-04,52,0.0,52.0,52
2016-01-05,63,0.0,55.0,55


In [17]:
df.fillna(0, inplace=True)
df = df.apply(lambda x : x.astype(int))
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 846 entries, 2016-01-01 to 2018-04-30
Data columns (total 4 columns):
Pedidos        846 non-null int32
Soler Dia      846 non-null int32
Soler Noche    846 non-null int32
Totales        846 non-null int32
dtypes: int32(4)
memory usage: 59.8 KB


In [28]:
df.to_excel("df total vs pred.xls")

# Aplicando el Proceso

In [29]:
df = pd.read_excel("df total vs pred.xls")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 5 columns):
Fecha             846 non-null datetime64[ns]
PedidosReales     846 non-null int64
SolerDiaPred      829 non-null float64
SolerNochePred    829 non-null float64
TotalesPred       846 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 33.1 KB


## 1) Feature Creation

In [30]:
df["difRealPred"] = df["PedidosReales"] - df["TotalesPred"]
df.describe()

,PedidosReales,SolerDiaPred,SolerNochePred,TotalesPred,difRealPred
count,846.000000,829.000000,829.000000,846.000000,846.000000
mean,99.589835,29.749095,79.039807,106.602837,-7.013002
std,31.761252,18.416734,25.900863,38.479850,27.885838
min,8.000000,0.000000,0.000000,0.000000,-133.000000
25%,74.000000,20.000000,60.000000,78.000000,-20.000000
50%,96.000000,27.000000,73.000000,104.000000,-6.000000
75%,125.000000,35.000000,98.000000,135.000000,5.000000
max,179.000000,146.000000,180.000000,247.000000,140.000000


In [32]:
df["difAbsRealPred"] = df["difRealPred"].abs()
df.describe()

,PedidosReales,SolerDiaPred,SolerNochePred,TotalesPred,difRealPred,difAbsRealPred
count,846.000000,829.000000,829.000000,846.000000,846.000000,846.000000
mean,99.589835,29.749095,79.039807,106.602837,-7.013002,19.256501
std,31.761252,18.416734,25.900863,38.479850,27.885838,21.344992
min,8.000000,0.000000,0.000000,0.000000,-133.000000,0.000000
25%,74.000000,20.000000,60.000000,78.000000,-20.000000,6.000000
50%,96.000000,27.000000,73.000000,104.000000,-6.000000,13.000000
75%,125.000000,35.000000,98.000000,135.000000,5.000000,25.000000
max,179.000000,146.000000,180.000000,247.000000,140.000000,140.000000


In [33]:
df.to_excel("1 - FC - df total vs pred.xls")